In [1]:
#hide
import core_constants as cc
import functions as fx
import requests
from bs4 import BeautifulSoup
import lxml
import time
import json
import pandas as pd
import sqlite3 as sql
import recordlinkage
import csv

In [28]:
def normalizeAACollege(recruitSchool, schoolsJSON):
    college = ""
    for school in schoolsJSON:
        if ('wikipedia' in school.keys()):
            recruitSchool = recruitSchool.lower().lstrip().rstrip()
            if (recruitSchool == school['wikipedia']):
                college = school['id']
                break
    print(college)
    if college != '':
        return college
    else:
        return recruitSchool

def handle_allAmerican(years, headers, sleepyTime=5):
    all_players = []
    for y in years: 
        url = 'https://en.wikipedia.org/wiki/{}_College_Football_All-America_Team'.format(y)
        r = requests.get(url, headers=headers)
        aaSoup = BeautifulSoup(r.text, 'lxml')
        for x in aaSoup.find_all("li", class_=""):
            try:
                player = []
                if ((",") in x.text and ("(") in x.text and (")") in x.text and ("{") not in x.text \
                    and ("Archived") not in x.text and ("Gridiron") not in x.text and ("edited") not in x.text \
                    and ("all-purpose") not in x.text and ("Football") not in x.text and ("Foundation") not in x.text):
                    playerInfo = x.text.split(",", 1)
                    #year
                    player.append(int(y))
                    #Name
                    player.append(playerInfo[0])
                    playerLocationAwards = playerInfo[1].split("(",1)

                    #School
                    if ("CONSENSUS" in x.text):
                        playerLocationAwards[0] = playerLocationAwards[0].replace("-- CONSENSUS --", "")
                    elif ("UNANIMOUS" in x.text):
                        playerLocationAwards[0] = playerLocationAwards[0].replace("-- UNANIMOUS --", "")
                    playerLocationAwards[0] = playerLocationAwards[0].lower()
                    if ('miami' in playerLocationAwards[0]):
                        playerLocationAwards[0] = 'miamifl'
                    player.append(playerLocationAwards[0].strip().replace(',',''))
                    #Awards String
                    awardString = playerLocationAwards[1]
                    #coaches (AFCA)
                    if ("AFCA" in awardString):
                        player.append(1)
                    else:
                        player.append(0)
                    #associated press (AP)
                    if ("AP" in awardString):
                        player.append(1)
                    else:
                        player.append(0)
                    #writers (FWAA)
                    if ("FWAA" in awardString):
                        player.append(1)
                    else:
                        player.append(0)
                    #sporting news (TSN)
                    if ("TSN" in awardString):
                        player.append(1)
                    else:
                        player.append(0)
                    #walter camp (WCFF)
                    if ("WCFF" in awardString):
                        player.append(1)
                    else:
                        player.append(0)
                    all_players.append(player)
            except:
                print('Error: ' + x)
    time.sleep(sleepyTime)

    final_aaSelections = []
    aa_keys = ['Year', 'PlayerName', 'College', 'AllAmericanAFCA', 'AllAmericanAP', 'AllAmericanFWAA', 'AllAmericanTSN', 'AllAmericanWCFF']

    for list in all_players:
        newdict = {aa_keys[i]: list[i] for i in range(len(aa_keys))}
        newdict['College'] = normalizeAACollege(newdict['College'], cc.get_schoolsList())

        final_aaSelections.append(newdict)
        newdict = {}
    
    
    return final_aaSelections

In [32]:
years = ['2013']
all_players = handle_allAmerican(years, cc.get_header())

alabama
florida-state
arizona
auburn
boston-college
florida-state
oregon-state
texas-am
vanderbilt
penn-state
clemson
texas-tech
north-carolina
florida-state
oregon
oklahoma
alabama
michigan
texas-am
ohio-state
baylor
florida-state
arkansas
stanford
clemson
south-carolina
pittsburgh
texas
florida-state
south-carolina
missouri
louisville
arizona-state
usc
ucla
wisconsin
buffalo
alabama
stanford
ohio-state
florida-state
washington-state
alabama
michigan-state
baylor
oregon
oklahoma-state
virginia
florida-state
ole-miss
stanford
tcu
northern-illinois
florida-state
northwestern
texas
boston-college
memphis
texas-am

michigan-state
western-kentucky
lsu
auburn
ucla
northern-illinois
stanford
north-carolina


In [33]:
all_players

[{'Year': 2013,
  'PlayerName': 'A. J. McCarron',
  'College': 'alabama',
  'AllAmericanAFCA': 1,
  'AllAmericanAP': 0,
  'AllAmericanFWAA': 0,
  'AllAmericanTSN': 0,
  'AllAmericanWCFF': 1},
 {'Year': 2013,
  'PlayerName': 'Jameis Winston',
  'College': 'florida-state',
  'AllAmericanAFCA': 0,
  'AllAmericanAP': 1,
  'AllAmericanFWAA': 1,
  'AllAmericanTSN': 1,
  'AllAmericanWCFF': 0},
 {'Year': 2013,
  'PlayerName': "Ka'Deem Carey",
  'College': 'arizona',
  'AllAmericanAFCA': 1,
  'AllAmericanAP': 1,
  'AllAmericanFWAA': 1,
  'AllAmericanTSN': 0,
  'AllAmericanWCFF': 1},
 {'Year': 2013,
  'PlayerName': 'Tre Mason',
  'College': 'auburn',
  'AllAmericanAFCA': 0,
  'AllAmericanAP': 0,
  'AllAmericanFWAA': 0,
  'AllAmericanTSN': 1,
  'AllAmericanWCFF': 0},
 {'Year': 2013,
  'PlayerName': 'Andre Williams',
  'College': 'boston-college',
  'AllAmericanAFCA': 1,
  'AllAmericanAP': 1,
  'AllAmericanFWAA': 1,
  'AllAmericanTSN': 1,
  'AllAmericanWCFF': 1},
 {'Year': 2013,
  'PlayerName': 'K